# Stocastic Petri Net Analysis

- Evaluation for a stochastic model described by SPN/GSPN/MRSPN
- The tool `gospn` creates a marking graph (CTMC transition matrices)
- NMarkov uses the CTMC

## Install for gospn

- The repository: [https://github.com/JuliaReliab/gospn](https://github.com/JuliaReliab/gospn)
- Build with Go or download a binary directly from [https://github.com/JuliaReliab/gospn/releases/latest](https://github.com/JuliaReliab/gospn/releases/latest)
- Put the binary `gospn` to the directory

In [ ]:
;wget https://github.com/JuliaReliab/gospn/releases/download/v0.10.2/gospn-0.10.2-linux-amd64.tar.gz

In [ ]:
;tar zxvf gospn-0.10.2-linux-amd64.tar.gz

## Install for NMarkov

https://github.com/JuliaReliab/NMarkov.jl

In [ ]:
using Pkg
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/Origin.jl.git"))
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/Deformula.jl.git"))
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/SparseMatrix.jl.git"))
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/NMarkov.jl.git"))

## Install Graphviz

In [ ]:
;sudo apt-get update -y

In [ ]:
;sudo apt-get install -y graphviz

In [ ]:
using Pkg
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/JuliaDot.jl.git"))

## Install Others

In [ ]:
# require the following packages.
# Install them with the package mode.

using Pkg
Pkg.add("MAT")
Pkg.add("JSON")
Pkg.add("Plots")
Pkg.add("SparseArrays")
Pkg.add("Distributions")

## Initialize

Load packages

In [ ]:
using Origin
using SparseMatrix
using NMarkov
using SparseArrays
using Plots
using MAT
using JSON
using Distributions
using JuliaDot

## Example: GSPN

Use the model: IaaS Cloud (monolithic)
```
  R. Ghosh, F. Longo, F. Frattini, S. Russo and K.S. Trivedi,
  Scalable analytics for IaaS cloud availability,
  IEEE Transactions on Cloud Computing, 2:1, 57-70, 2014.
```

### Draw a petrinet

In [ ]:
run(`./gospn view -i iaas_cloud.spn -o tmp.dot`)
drawfile("tmp.dot")

### Generate the marking graph

In [ ]:
run(`./gospn mark -i iaas_cloud.spn -o result.mat -t -g gmark.dot`)
matfile = matopen("result.mat")

In [ ]:
drawfile("gmark.dot")

In the above graph, Gxx indicates EXP/GEN group where there is no enabled IMM transitions. Also Ixx indicates IMM group where there is one or more enabled IMM transitions. From the above graph, we find that there is no self-loop in I0. Then I0 can be vanished by
```
Q = G0G0E + G0I0E * I0G0I
```

In [ ]:
# Read matrices
G0G0E = read(matfile, "G0G0E")
G0I0E = read(matfile, "G0I0E")
I0G0I = read(matfile, "I0G0I");

In [ ]:
# CTMC transition matrix (IMM group should be vanished)
Q = G0G0E + G0I0E * I0G0I;
println("nnz ", length(Q.nzval) - Q.m)

In [ ]:
# initial probabiliyt vector
x0 = read(matfile, "initG0");

In [ ]:
# load reward vector
r1 = read(matfile, "rwd1G0"); #Ph
r2 = read(matfile, "rwd2G0"); #Pw
r3 = read(matfile, "rwd3G0"); #Pc

In [ ]:
# expected number of hot/warm/cold servers in steady state
pis, conv, = stgs(Q)
println("Convergence ", conv)
println("# of hot servers ", sum(pis .* r1))
println("# of warm servers ", sum(pis .* r2))
println("# of cold servers ", sum(pis .* r3))

In [ ]:
# Transient
ts = LinRange(0.0, 1000.0, 100)
irwd1, crwd1, xt1, cxt1 = tran(Q, x0, r1, ts, forward=:T);
irwd2, crwd2, xt2, cxt2 = tran(Q, x0, r2, ts, forward=:T);
irwd3, crwd3, xt3, cxt3 = tran(Q, x0, r3, ts, forward=:T);

In [ ]:
plot(ts, [irwd1, irwd2, irwd3])

In [ ]:
println("# of hot servers  (time average) ", sum(cxt1 .* r1)/1000.0)
println("# of warm servers (time average) ", sum(cxt1 .* r2)/1000.0)
println("# of cold servers (time average) ", sum(cxt1 .* r3)/1000.0)

## Example: MRSPN

Use the model: RAID6
```
  F. Machida, R. Xia and K.S. Trivedi,
  Performability modeling for RAID storage systems by Markov regenerative process,
  IEEE Transactions on Dependable and Secure Computing
```

### Draw a Petrinet

In [ ]:
run(`./gospn view -i raid6.spn -o tmp.dot`)
drawfile("tmp.dot")

### Generate the marking graph

In [ ]:
run(`./gospn mark -i raid6.spn -o result.mat -t -g gmark.dot`)
matfile = matopen("result.mat")

In [ ]:
# Draw a group grapah
drawfile("gmark.dot")

This model includes two GEN transitions; Trecon and Trebuild. Thus there are three groups for markings
- G0: There is no enabled GEN transition
- G1: Trecon is enabled (# of states is 1)
- G2: Trebuild is enabled (# of states is 2)
Now we focus on the discrete-time Markov chain at time instants when G1 and G2 enters, so-called embedded Markov chain (EMC). G1 is 

In [ ]:
# Read matrices
G0G0E = read(matfile, "G0G0E")
G0G2E = read(matfile, "G0G2E")
G1G1E = read(matfile, "G1G1E")
G1G0P1 = read(matfile, "G1G0P1")
G2G0P0 = read(matfile, "G2G0P0")
G2G2E = read(matfile, "G2G2E")
G2G2P0 = read(matfile, "G2G2P0")
G2I2E = read(matfile, "G2I2E")
I2G1I = read(matfile, "I2G1I");

In [ ]:
# parameter for GEN dist
MTTR1 = 2.0 # [hours]
MTTR2 = 24.0; # [hours] reconfigure

In [ ]:
# Make EMC

## Matrix on time instant of the end of state
V0 = -G0G0E \ eye(G0G0E)
V1, V1c = mexpc(G1G1E, eye(G1G1E), MTTR2) # constant distribution
V2, V2c = mexpc(G2G2E, eye(G2G2E), MTTR1) # constant distribution

# indicies when all states are concatinated [V0, V1, V2]
indices = [1:1, 2:2, 3:4] 

## Transition probability matrices for EMC.
## This is constucted by the groupmark graph
P = spzeros(AbstractMatrix{Float64}, 3, 3) # blockmatrix
@origin P=>0 begin
    P[0,2] = V0 * G0G2E
    P[1,0] = V1 * G1G0P1
    P[2,0] = V2 * G2G0P0
    P[2,1] = V2 * G2I2E * I2G1I
    P[2,2] = V2 * G2G2P0
end
P=sparse(block(P))

In [ ]:
# stationary vector for EMC
pid = gth(Matrix(P))

In [ ]:
# sojourn time & stationary distribution
S = spzeros(AbstractMatrix{Float64}, 3, 3) # blockmatrix
@origin S=>0 begin
    S[0,0] = V0
    S[1,1] = V1c
    S[2,2] = V2c
end
S=sparse(block(S))
sojourn = S' * pid
pis = sojourn / sum(sojourn)

In [ ]:
# Read reward
availG0 = read(matfile, "availG0")
availG1 = read(matfile, "availG1")
availG2 = read(matfile, "availG2")
unavailG0 = read(matfile, "unavailG0")
unavailG1 = read(matfile, "unavailG1")
unavailG2 = read(matfile, "unavailG2");

In [ ]:
@origin indices=>0 begin
    avail = sum(pis[indices[0]] .* availG0) + sum(pis[indices[1]] .* availG1) + sum(pis[indices[2]] .* availG2)
    unavail = sum(pis[indices[0]] .* unavailG0) + sum(pis[indices[1]] .* unavailG1) + sum(pis[indices[2]] .* unavailG2)
end
MTTDL = avail / unavail * MTTR2 / 24 /365

## Example: MC simulation

Use the model: IaaS Cloud (monolithic)
```
  R. Ghosh, F. Longo, F. Frattini, S. Russo and K.S. Trivedi,
  Scalable analytics for IaaS cloud availability,
  IEEE Transactions on Cloud Computing, 2:1, 57-70, 2014.
```

### Draw a petrinet

In [ ]:
run(`./gospn view -i iaas_cloud.spn -o tmp.dot`)
drawfile("tmp.dot")

In [ ]:
## generate config for simulation with JSON
config = Dict(
    "time" => 1000.0,   # the stop condition for one simulation
    "firings" => 0,       # the stop condition for one simulation
    "simulations" => 1000, # the number of simulation runs
    "rewards" => ["rwd1", "rwd2", "rwd3"]
)

open("config.json", "w") do f 
    write(f, JSON.json(config)) 
end;

In [ ]:
run(`./gospn sim -i iaas_cloud.spn -o result.mat -f config.json -s 12345`)
matfile = matopen("result.mat")
keys(read(matfile))

In [ ]:
elapsedtime = read(matfile, "elapsedtime")
rwd1 = read(matfile, "rwd1_crwd") ./ elapsedtime
rwd2 = read(matfile, "rwd2_crwd") ./ elapsedtime
rwd3 = read(matfile, "rwd3_crwd") ./ elapsedtime;

In [ ]:
# utility
function meanConfidenceInterval(data, alpha)
    n = length(data)
    m = sum(data) / n
    s = sqrt(sum((data .- m).^2) / (n-1))
    h = quantile(TDist(n-1), 1-(1-alpha)/2) * s / sqrt(n)
    (m, m-h, m+h)
end

In [ ]:
println(meanConfidenceInterval(rwd1, 0.99))
println(meanConfidenceInterval(rwd2, 0.99))
println(meanConfidenceInterval(rwd3, 0.99))

In [ ]:
# Results from CTMC analysis
# rwd1: the number of hot servers  (time average) 2.9970927984146303
# rwd2: the number of warm servers (time average) 2.9796356056398428
# rwd3: the number of cold servers (time average) 2.991510485351518